# Sales Forecasting Analysis

This notebook demonstrates multiple time series forecasting techniques for retail sales data:
- **ARIMA**: Traditional autoregressive integrated moving average model
- **Prophet**: Facebook's robust forecasting library for business time series
- **XGBoost**: Machine learning approach for time series prediction

## Business Objective
Predict future sales to optimize inventory management, staffing, and marketing spend.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Time series specific libraries
# Note: Install these in your local environment:
# pip install statsmodels prophet xgboost scikit-learn
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Generate sample retail sales data
def generate_sales_data():
    """
    Generate synthetic retail sales data with seasonality and trends
    """
    date_range = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
    n_days = len(date_range)
    
    # Base trend
    trend = np.linspace(1000, 1500, n_days)
    
    # Seasonal patterns
    yearly_seasonality = 200 * np.sin(2 * np.pi * np.arange(n_days) / 365.25)
    weekly_seasonality = 100 * np.sin(2 * np.pi * np.arange(n_days) / 7)
    
    # Holiday spikes
    holiday_boost = np.zeros(n_days)
    for year in range(2020, 2024):
        # Black Friday boost
        black_friday = pd.Timestamp(f'{year}-11-25') + pd.Timedelta(days=(3-pd.Timestamp(f'{year}-11-25').weekday())%7)
        if black_friday in date_range:
            idx = date_range.get_loc(black_friday)
            holiday_boost[idx-2:idx+3] += 500
    
    # Random noise
    noise = np.random.normal(0, 50, n_days)
    
    # Combine components
    sales = trend + yearly_seasonality + weekly_seasonality + holiday_boost + noise
    sales = np.maximum(sales, 0)  # Ensure no negative sales
    
    return pd.DataFrame({
        'date': date_range,
        'sales': sales,
        'day_of_week': date_range.day_name(),
        'month': date_range.month,
        'year': date_range.year
    })

# Generate and prepare data
df = generate_sales_data()
df.set_index('date', inplace=True)
print(f"Dataset shape: {df.shape}")
df.head()

## Exploratory Data Analysis

In [ ]:
# Plot time series
plt.figure(figsize=(15, 10))

# Main time series
plt.subplot(2, 2, 1)
plt.plot(df.index, df['sales'])
plt.title('Daily Sales Over Time')
plt.ylabel('Sales ($)')

# Seasonal decomposition
decomposition = seasonal_decompose(df['sales'], model='additive', period=365)

plt.subplot(2, 2, 2)
plt.plot(decomposition.trend)
plt.title('Trend Component')
plt.ylabel('Sales ($)')

plt.subplot(2, 2, 3)
plt.plot(decomposition.seasonal[:365])
plt.title('Seasonal Component (1 Year)')
plt.ylabel('Sales ($)')

plt.subplot(2, 2, 4)
plt.plot(decomposition.resid)
plt.title('Residual Component')
plt.ylabel('Sales ($)')

plt.tight_layout()
plt.show()

# Summary statistics
print("\nSales Summary Statistics:")
print(df['sales'].describe())

## Model 1: ARIMA Forecasting

In [ ]:
# Split data for training and testing
train_size = int(len(df) * 0.8)
train_data = df['sales'][:train_size]
test_data = df['sales'][train_size:]

print(f"Training period: {train_data.index[0]} to {train_data.index[-1]}")
print(f"Testing period: {test_data.index[0]} to {test_data.index[-1]}")

# Fit ARIMA model
# Note: In practice, use auto_arima for parameter optimization
arima_model = ARIMA(train_data, order=(5, 1, 2))
arima_fitted = arima_model.fit()

# Generate forecasts
arima_forecast = arima_fitted.forecast(steps=len(test_data))
arima_forecast_df = pd.DataFrame({
    'date': test_data.index,
    'actual': test_data.values,
    'forecast': arima_forecast
})

# Calculate metrics
arima_mae = mean_absolute_error(test_data, arima_forecast)
arima_rmse = np.sqrt(mean_squared_error(test_data, arima_forecast))

print(f"\nARIMA Model Performance:")
print(f"MAE: ${arima_mae:.2f}")
print(f"RMSE: ${arima_rmse:.2f}")

## Model 2: Prophet Forecasting

In [ ]:
# Prepare data for Prophet
prophet_train = train_data.reset_index().rename(columns={'date': 'ds', 'sales': 'y'})
prophet_test = test_data.reset_index().rename(columns={'date': 'ds', 'sales': 'y'})

# Initialize and fit Prophet model
prophet_model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    changepoint_prior_scale=0.05
)

# Add custom seasonalities for retail
prophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

prophet_model.fit(prophet_train)

# Create future dataframe and predict
future = prophet_model.make_future_dataframe(periods=len(test_data))
prophet_forecast = prophet_model.predict(future)

# Extract test period forecasts
prophet_test_forecast = prophet_forecast.tail(len(test_data))['yhat'].values

# Calculate metrics
prophet_mae = mean_absolute_error(test_data, prophet_test_forecast)
prophet_rmse = np.sqrt(mean_squared_error(test_data, prophet_test_forecast))

print(f"\nProphet Model Performance:")
print(f"MAE: ${prophet_mae:.2f}")
print(f"RMSE: ${prophet_rmse:.2f}")

# Plot Prophet components
fig = prophet_model.plot_components(prophet_forecast)
plt.show()

## Model 3: XGBoost Forecasting

In [ ]:
# Feature engineering for XGBoost
def create_features(data):
    """
    Create time-based features for XGBoost
    """
    df_features = data.copy()
    
    # Time-based features
    df_features['day'] = df_features.index.day
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    
    # Lag features
    for lag in [1, 7, 14, 30]:
        df_features[f'sales_lag_{lag}'] = df_features['sales'].shift(lag)
    
    # Rolling statistics
    for window in [7, 14, 30]:
        df_features[f'sales_rolling_mean_{window}'] = df_features['sales'].rolling(window=window).mean()
        df_features[f'sales_rolling_std_{window}'] = df_features['sales'].rolling(window=window).std()
    
    # Weekend indicator
    df_features['is_weekend'] = df_features['dayofweek'].isin([5, 6]).astype(int)
    
    return df_features.dropna()

# Create features
df_features = create_features(df)

# Split with features
train_size_xgb = int(len(df_features) * 0.8)
X_train = df_features.drop('sales', axis=1)[:train_size_xgb]
y_train = df_features['sales'][:train_size_xgb]
X_test = df_features.drop('sales', axis=1)[train_size_xgb:]
y_test = df_features['sales'][train_size_xgb:]

# Train XGBoost model
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

xgb_model.fit(X_train, y_train)

# Generate predictions
xgb_forecast = xgb_model.predict(X_test)

# Calculate metrics
xgb_mae = mean_absolute_error(y_test, xgb_forecast)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_forecast))

print(f"\nXGBoost Model Performance:")
print(f"MAE: ${xgb_mae:.2f}")
print(f"RMSE: ${xgb_rmse:.2f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

## Model Comparison and Visualization

In [ ]:
# Compare all models
comparison_results = pd.DataFrame({
    'Model': ['ARIMA', 'Prophet', 'XGBoost'],
    'MAE': [arima_mae, prophet_mae, xgb_mae],
    'RMSE': [arima_rmse, prophet_rmse, xgb_rmse]
})

print("Model Performance Comparison:")
print(comparison_results)

# Visualization
plt.figure(figsize=(15, 8))
plt.plot(test_data.index, test_data.values, label='Actual Sales', linewidth=2)
plt.plot(test_data.index, arima_forecast, label='ARIMA Forecast', alpha=0.7)
plt.plot(test_data.index, prophet_test_forecast, label='Prophet Forecast', alpha=0.7)
plt.plot(y_test.index, xgb_forecast, label='XGBoost Forecast', alpha=0.7)

plt.title('Sales Forecasting Model Comparison')
plt.xlabel('Date')
plt.ylabel('Sales ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Generate 30-day forecast with best model
best_model_name = comparison_results.loc[comparison_results['MAE'].idxmin(), 'Model']
print(f"\nBest performing model: {best_model_name}")

# Future forecast (using Prophet for example)
future_30_days = prophet_model.make_future_dataframe(periods=30)
future_forecast = prophet_model.predict(future_30_days)
next_30_days = future_forecast.tail(30)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

print("\n30-Day Sales Forecast:")
print(next_30_days.head(10))

## Business Recommendations

Based on the forecasting analysis:

1. **Inventory Planning**: Use 30-day forecasts to optimize stock levels
2. **Staffing**: Align workforce with predicted demand patterns
3. **Marketing Budget**: Allocate spending based on forecasted sales periods
4. **Model Monitoring**: Retrain models monthly with new data